# 📊 Canvas Course-Level Analytics - Ranking Analysis

**Objetivo:** Analizar métricas de engagement a nivel de **CURSO** para identificar cursos con alto potencial para el sistema de alerta temprana.

## Diferencia con análisis de cuenta
- **Antes:** Agregado a nivel de cuenta/programa (PREGRADO, Providencia, etc.)
- **Ahora:** Métricas individuales por curso (86005, 86676, etc.)

## Métricas a Nivel de Curso
- **Enrollments:** `n_students`, `grade_variance`, `pass_rate`
- **Student Summaries:** `page_views`, `participations`, `tardiness`
- **Resources:** `n_assignments`, `n_modules`, `n_quizzes`
- **Assignment Analytics:** `missing_rate`, score distributions

## Criterios de Ranking (para Early Warning)
1. **Grade Variance** - Mayor varianza = mejor para predicción
2. **Pass Rate** - Ideal entre 20-80% 
3. **LMS Design** - Más recursos = más señales de engagement
4. **Student Activity** - Page views y participations por estudiante

---

## 1. Setup y Configuración

In [1]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from datetime import datetime
import warnings
import time
import json

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Cargar credenciales
load_dotenv()

API_URL = os.getenv('CANVAS_API_URL')
API_TOKEN = os.getenv('CANVAS_API_TOKEN')
headers = {'Authorization': f'Bearer {API_TOKEN}'}

print(f"✅ API URL: {API_URL}")
print(f"✅ Token configurado: {'Sí' if API_TOKEN else 'No'}")

✅ API URL: https://uautonoma.test.instructure.com
✅ Token configurado: Sí


## 2. Funciones de API para Analytics a Nivel de Curso

In [ ]:
# ============================================================
# FUNCIONES DE API - NIVEL DE CURSO
# ============================================================

def get_courses_from_account(account_id, term_id=None, min_students=10):
    """
    Obtiene cursos de una cuenta con filtro de estudiantes mínimos.
    
    GET /api/v1/accounts/{account_id}/courses
    """
    params = {
        'per_page': 100,
        'include[]': ['total_students', 'term'],
        'with_enrollments': True
    }
    if term_id:
        params['enrollment_term_id'] = term_id
    
    courses = []
    url = f'{API_URL}/api/v1/accounts/{account_id}/courses'
    
    while url:
        r = requests.get(url, headers=headers, params=params)
        if r.status_code != 200:
            print(f"Error {r.status_code} fetching courses from account {account_id}")
            break
        
        for course in r.json():
            if course.get('total_students', 0) >= min_students:
                courses.append({
                    'course_id': course['id'],
                    'course_name': course['name'],
                    'total_students': course.get('total_students', 0),
                    'term_name': course.get('term', {}).get('name', 'Unknown'),
                    'term_id': course.get('enrollment_term_id'),
                    'account_id': account_id
                })
        
        # Pagination via Link header
        url = r.links.get('next', {}).get('url')
        params = {}  # URL already contains params
        time.sleep(0.2)
    
    return sorted(courses, key=lambda x: x['total_students'], reverse=True)


def get_course_enrollments_with_grades(course_id):
    """
    Obtiene enrollments con calificaciones.
    
    GET /api/v1/courses/{course_id}/enrollments
    """
    r = requests.get(
        f'{API_URL}/api/v1/courses/{course_id}/enrollments',
        headers=headers,
        params={
            'type[]': 'StudentEnrollment',
            'per_page': 100,
            'include[]': ['grades', 'total_scores']
        }
    )
    if r.status_code != 200:
        return []
    return r.json()


def get_course_student_summaries(course_id):
    """
    Obtiene métricas de actividad por estudiante.
    
    GET /api/v1/courses/{course_id}/analytics/student_summaries
    """
    r = requests.get(
        f'{API_URL}/api/v1/courses/{course_id}/analytics/student_summaries',
        headers=headers,
        params={'per_page': 100}
    )
    if r.status_code != 200:
        return []
    return r.json()


def get_course_assignments(course_id):
    """
    Obtiene lista de assignments del curso.
    
    GET /api/v1/courses/{course_id}/assignments
    """
    r = requests.get(
        f'{API_URL}/api/v1/courses/{course_id}/assignments',
        headers=headers,
        params={'per_page': 100}
    )
    if r.status_code != 200:
        return []
    return r.json()


def get_course_modules(course_id):
    """
    Obtiene módulos del curso.
    
    GET /api/v1/courses/{course_id}/modules
    """
    r = requests.get(
        f'{API_URL}/api/v1/courses/{course_id}/modules',
        headers=headers,
        params={'per_page': 100}
    )
    if r.status_code != 200:
        return []
    return r.json()


def get_course_quizzes(course_id):
    """
    Obtiene quizzes del curso.
    
    GET /api/v1/courses/{course_id}/quizzes
    """
    r = requests.get(
        f'{API_URL}/api/v1/courses/{course_id}/quizzes',
        headers=headers,
        params={'per_page': 100}
    )
    if r.status_code != 200:
        return []
    return r.json()


def get_course_assignment_analytics(course_id):
    """
    Obtiene estadísticas de assignments (quartiles, tardiness).
    
    GET /api/v1/courses/{course_id}/analytics/assignments
    """
    r = requests.get(
        f'{API_URL}/api/v1/courses/{course_id}/analytics/assignments',
        headers=headers
    )
    if r.status_code != 200:
        return []
    return r.json()


def get_enrollment_terms(account_id):
    """
    Obtiene términos de matrícula disponibles.
    """
    url = f"{API_URL}/api/v1/accounts/{account_id}/terms"
    response = requests.get(url, headers=headers, params={'per_page': 50})
    if response.status_code == 200:
        return response.json().get('enrollment_terms', [])
    return []


print("✅ Funciones de API a nivel de curso definidas")

## 3. Configuración de Cuentas y Términos

In [ ]:
# Obtener términos disponibles
terms = get_enrollment_terms(1)

print("📅 TÉRMINOS DISPONIBLES")
print("=" * 50)
terms_df = pd.DataFrame(terms)
if not terms_df.empty:
    display(terms_df[['id', 'name']].head(10))
    
# Términos relevantes para análisis
TERM_IDS = {
    '1er_sem_2025': 322,
    '2do_sem_2025': 336,
    'bimestral_2025': 352
}
print(f"\n📌 Términos principales: {TERM_IDS}")

# Término activo para análisis
ACTIVE_TERM_ID = 336  # 2do Semestre 2025
print(f"🎯 Término activo para análisis: {ACTIVE_TERM_ID}")

In [ ]:
# Cuentas a escanear para obtener cursos
ACCOUNTS_TO_SCAN = {
    'Ing_Control_Gestion': 719,      # Ya analizado - control
    'Ing_Control_Gestion_Alt': 718,  # Alternativo
    'Ing_Civil_Industrial': 730,     # Ingeniería
}

# Para análisis más amplio, descomentar:
# ACCOUNTS_TO_SCAN = {
#     'Providencia': 176,              # Sede completa
#     'Ing_Control_Gestion': 719,
#     'Campus_Virtual_PG': 401,        # Postgrado
# }

print("🏛️ CUENTAS A ESCANEAR")
print("=" * 50)
for name, acc_id in ACCOUNTS_TO_SCAN.items():
    print(f"  {acc_id}: {name}")

# Configuración de filtros
MIN_STUDENTS = 15  # Mínimo de estudiantes por curso
print(f"\n📊 Filtro: cursos con >= {MIN_STUDENTS} estudiantes")

## 4. Descubrimiento y Análisis de Cursos

Obtiene cursos de las cuentas configuradas y extrae métricas a nivel de curso.

In [ ]:
def analyze_course(course_id, course_name=None):
    """
    Analiza un curso individual y extrae todas las métricas relevantes.
    
    Retorna dict con:
    - Métricas de calificaciones (grade_variance, pass_rate)
    - Métricas de actividad (page_views, participations)
    - Métricas de diseño LMS (n_assignments, n_modules, n_quizzes)
    - Métricas de tardiness (on_time_rate, missing_rate)
    """
    result = {
        'course_id': course_id,
        'course_name': course_name or f"Course {course_id}",
        
        # Grade metrics
        'n_students_with_grades': 0,
        'grade_mean': None,
        'grade_std': None,
        'grade_min': None,
        'grade_max': None,
        'pass_rate': None,
        
        # Activity metrics (aggregated from student summaries)
        'total_page_views': 0,
        'total_participations': 0,
        'avg_page_views': 0,
        'avg_participations': 0,
        
        # Tardiness metrics
        'avg_on_time_rate': None,
        'avg_missing_rate': None,
        
        # LMS Design metrics
        'n_assignments': 0,
        'n_modules': 0,
        'n_quizzes': 0,
        
        # Assignment analytics
        'assignments_with_scores': 0,
        'avg_assignment_missing_rate': None,
        
        # Status
        'has_grades': False,
        'has_activity': False,
        'recommendation': 'SKIP'
    }
    
    # 1. Get enrollments with grades
    enrollments = get_course_enrollments_with_grades(course_id)
    grades = []
    for e in enrollments:
        final_score = e.get('grades', {}).get('final_score')
        if final_score is not None:
            grades.append(final_score)
    
    if len(grades) >= 5:  # Need at least 5 students with grades
        result['has_grades'] = True
        result['n_students_with_grades'] = len(grades)
        result['grade_mean'] = np.mean(grades)
        result['grade_std'] = np.std(grades)
        result['grade_min'] = np.min(grades)
        result['grade_max'] = np.max(grades)
        result['pass_rate'] = sum(1 for g in grades if g >= 57) / len(grades)
    
    # 2. Get student summaries (activity)
    summaries = get_course_student_summaries(course_id)
    if summaries:
        result['has_activity'] = True
        
        page_views = [s.get('page_views', 0) for s in summaries]
        participations = [s.get('participations', 0) for s in summaries]
        
        result['total_page_views'] = sum(page_views)
        result['total_participations'] = sum(participations)
        result['avg_page_views'] = np.mean(page_views) if page_views else 0
        result['avg_participations'] = np.mean(participations) if participations else 0
        
        # Tardiness breakdown
        on_time_rates = []
        missing_rates = []
        for s in summaries:
            tb = s.get('tardiness_breakdown', {})
            total = tb.get('on_time', 0) + tb.get('late', 0) + tb.get('missing', 0)
            if total > 0:
                on_time_rates.append(tb.get('on_time', 0) / total)
                missing_rates.append(tb.get('missing', 0) / total)
        
        if on_time_rates:
            result['avg_on_time_rate'] = np.mean(on_time_rates)
            result['avg_missing_rate'] = np.mean(missing_rates)
    
    # 3. Get LMS resources count
    result['n_assignments'] = len(get_course_assignments(course_id))
    result['n_modules'] = len(get_course_modules(course_id))
    result['n_quizzes'] = len(get_course_quizzes(course_id))
    
    # 4. Get assignment analytics
    assignment_analytics = get_course_assignment_analytics(course_id)
    if assignment_analytics:
        result['assignments_with_scores'] = len([a for a in assignment_analytics if a.get('median') is not None])
        missing_rates = [a.get('tardiness_breakdown', {}).get('missing', 0) 
                        for a in assignment_analytics if a.get('tardiness_breakdown')]
        if missing_rates:
            result['avg_assignment_missing_rate'] = np.mean(missing_rates)
    
    # 5. Determine recommendation
    if result['has_grades'] and result['grade_std'] and result['grade_std'] > 10:
        if result['n_assignments'] >= 5 and result['pass_rate'] is not None:
            if 0.2 <= result['pass_rate'] <= 0.8:
                result['recommendation'] = 'HIGH POTENTIAL'
            else:
                result['recommendation'] = 'MEDIUM POTENTIAL'
        elif result['n_assignments'] >= 3:
            result['recommendation'] = 'MEDIUM POTENTIAL'
        else:
            result['recommendation'] = 'LOW - Few assignments'
    elif result['has_grades']:
        result['recommendation'] = 'LOW - Low grade variance'
    else:
        result['recommendation'] = 'SKIP - No grades'
    
    return result


print("✅ Función de análisis de curso definida")

In [ ]:
# Descubrir cursos de las cuentas configuradas
print("🔍 DESCUBRIENDO CURSOS")
print("=" * 60)

all_courses = []

for account_name, account_id in ACCOUNTS_TO_SCAN.items():
    print(f"\n📂 Escaneando {account_name} (ID: {account_id})...")
    courses = get_courses_from_account(account_id, term_id=ACTIVE_TERM_ID, min_students=MIN_STUDENTS)
    print(f"   Encontrados: {len(courses)} cursos con >= {MIN_STUDENTS} estudiantes")
    
    for c in courses:
        c['account_name'] = account_name
    all_courses.extend(courses)

print(f"\n📊 TOTAL: {len(all_courses)} cursos para analizar")

# Mostrar primeros cursos
if all_courses:
    courses_df = pd.DataFrame(all_courses)
    display(courses_df[['course_id', 'course_name', 'total_students', 'account_name']].head(15))

In [ ]:
# Analizar cada curso
print("📊 ANALIZANDO CURSOS")
print("=" * 60)

course_analyses = []

for i, course in enumerate(all_courses):
    course_id = course['course_id']
    course_name = course['course_name']
    
    print(f"  [{i+1}/{len(all_courses)}] {course_id}: {course_name[:40]}...", end=" ")
    
    try:
        analysis = analyze_course(course_id, course_name)
        analysis['total_students'] = course['total_students']
        analysis['account_name'] = course['account_name']
        analysis['term_id'] = course.get('term_id')
        course_analyses.append(analysis)
        
        status = "✅" if analysis['has_grades'] else "⚠️"
        print(f"{status} ({analysis['recommendation']})")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    time.sleep(0.5)  # Rate limiting

print(f"\n📈 Cursos analizados: {len(course_analyses)}")

In [ ]:
# Crear DataFrame con resultados
courses_df = pd.DataFrame(course_analyses)

# Mostrar resumen
print(f"📊 RESUMEN DE ANÁLISIS")
print("=" * 60)
print(f"Total cursos analizados: {len(courses_df)}")
print(f"Cursos con notas: {courses_df['has_grades'].sum()}")
print(f"Cursos con actividad: {courses_df['has_activity'].sum()}")

print("\n📈 Distribución por Recomendación:")
print(courses_df['recommendation'].value_counts().to_string())

# Mostrar columnas principales
display_cols = ['course_id', 'course_name', 'total_students', 'n_students_with_grades',
                'grade_std', 'pass_rate', 'n_assignments', 'recommendation']
available_cols = [c for c in display_cols if c in courses_df.columns]

print("\n📋 Cursos con Mayor Potencial:")
high_potential = courses_df[courses_df['recommendation'].str.contains('HIGH|MEDIUM', na=False)]
if not high_potential.empty:
    display(high_potential[available_cols].head(10))
else:
    print("No se encontraron cursos con alto potencial en esta búsqueda.")

## 5. Cálculo de Métricas de Ranking a Nivel de Curso

Métricas compuestas para determinar el potencial de cada curso para early warning.

In [ ]:
def calculate_course_ranking_metrics(df):
    """
    Calcula métricas de ranking específicas para Early Warning System.
    
    Criterios:
    1. Grade Variance - Mayor varianza = mejor predicción
    2. Pass Rate - Ideal 20-80% (no todos aprueban/reprueban)
    3. LMS Design Score - Más recursos = más señales
    4. Activity Score - Engagement estudiantil
    5. Data Quality - Completitud de datos
    """
    df = df.copy()
    
    # 1. Grade Variance Score (normalizado)
    grade_std = df['grade_std'].fillna(0)
    df['grade_variance_score'] = grade_std / grade_std.max() if grade_std.max() > 0 else 0
    
    # 2. Pass Rate Score (penaliza extremos 0% y 100%)
    # Óptimo: 50%, decae hacia los extremos
    pass_rate = df['pass_rate'].fillna(0.5)
    df['pass_rate_score'] = 1 - np.abs(pass_rate - 0.5) * 2  # 1 at 50%, 0 at 0% or 100%
    
    # 3. LMS Design Score (combinación de recursos)
    df['lms_design_score'] = (
        df['n_assignments'].fillna(0) * 2 +  # Assignments más importantes
        df['n_quizzes'].fillna(0) * 1.5 +
        df['n_modules'].fillna(0) * 1
    )
    max_lms = df['lms_design_score'].max()
    df['lms_design_score'] = df['lms_design_score'] / max_lms if max_lms > 0 else 0
    
    # 4. Activity Score (engagement per student)
    df['activity_score'] = (
        df['avg_page_views'].fillna(0) + 
        df['avg_participations'].fillna(0) * 10  # Participations más valiosas
    )
    max_activity = df['activity_score'].max()
    df['activity_score'] = df['activity_score'] / max_activity if max_activity > 0 else 0
    
    # 5. Data Quality Score
    df['data_quality_score'] = (
        df['has_grades'].astype(int) * 0.4 +
        df['has_activity'].astype(int) * 0.3 +
        (df['n_students_with_grades'] >= 20).astype(int) * 0.3
    )
    
    # 6. Composite Score for Early Warning
    # Weighted combination focusing on prediction potential
    df['ew_potential_score'] = (
        df['grade_variance_score'] * 0.30 +      # 30% - Varianza de notas
        df['pass_rate_score'] * 0.20 +           # 20% - Balance aprobados/reprobados
        df['lms_design_score'] * 0.20 +          # 20% - Diseño del curso
        df['activity_score'] * 0.15 +            # 15% - Actividad estudiantil
        df['data_quality_score'] * 0.15          # 15% - Calidad de datos
    )
    
    return df


# Calcular métricas solo para cursos con datos
if not courses_df.empty:
    courses_with_metrics = calculate_course_ranking_metrics(courses_df)
    
    print("📊 MÉTRICAS DE RANKING CALCULADAS")
    print("=" * 60)
    
    metric_cols = ['course_name', 'grade_variance_score', 'pass_rate_score', 
                   'lms_design_score', 'activity_score', 'ew_potential_score']
    available_cols = [c for c in metric_cols if c in courses_with_metrics.columns]
    
    # Top 10 por potencial
    top_courses = courses_with_metrics.nlargest(10, 'ew_potential_score')
    display(top_courses[available_cols].round(3))
else:
    print("⚠️ No hay cursos para calcular métricas")

## 6. Visualizaciones a Nivel de Curso

In [ ]:
# Configuración de figuras
fig_dir = '../data/analytics/'
os.makedirs(fig_dir, exist_ok=True)

if not courses_df.empty and 'ew_potential_score' in courses_with_metrics.columns:
    
    # 6.1 Distribución de Pass Rate
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Histograma de Pass Rate
    ax1 = axes[0]
    pass_rates = courses_with_metrics['pass_rate'].dropna()
    if len(pass_rates) > 0:
        ax1.hist(pass_rates, bins=10, color='steelblue', edgecolor='black', alpha=0.7)
        ax1.axvline(x=0.57, color='red', linestyle='--', label='Umbral aprobación (57%)')
        ax1.axvspan(0.2, 0.8, alpha=0.2, color='green', label='Rango ideal (20-80%)')
        ax1.set_xlabel('Pass Rate', fontsize=11)
        ax1.set_ylabel('Número de Cursos', fontsize=11)
        ax1.set_title('📊 Distribución de Tasa de Aprobación', fontsize=12, fontweight='bold')
        ax1.legend()
    
    # Plot 2: Histograma de Grade Variance
    ax2 = axes[1]
    grade_stds = courses_with_metrics['grade_std'].dropna()
    if len(grade_stds) > 0:
        ax2.hist(grade_stds, bins=10, color='coral', edgecolor='black', alpha=0.7)
        ax2.axvline(x=10, color='green', linestyle='--', label='Mínimo útil (σ=10)')
        ax2.set_xlabel('Desviación Estándar de Notas', fontsize=11)
        ax2.set_ylabel('Número de Cursos', fontsize=11)
        ax2.set_title('📈 Distribución de Varianza de Notas', fontsize=12, fontweight='bold')
        ax2.legend()
    
    plt.tight_layout()
    plt.savefig(f'{fig_dir}viz_01_course_distributions.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✅ Guardado: {fig_dir}viz_01_course_distributions.png")
else:
    print("⚠️ No hay datos suficientes para visualizar")

In [ ]:
# 6.2 Scatter: Grade Variance vs Pass Rate (identificar cursos ideales)
if not courses_df.empty and 'grade_std' in courses_with_metrics.columns:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Filtrar cursos con datos
    plot_data = courses_with_metrics[
        courses_with_metrics['grade_std'].notna() & 
        courses_with_metrics['pass_rate'].notna()
    ].copy()
    
    if len(plot_data) > 0:
        # Scatter plot
        scatter = ax.scatter(
            plot_data['pass_rate'],
            plot_data['grade_std'],
            s=plot_data['total_students'] * 3 + 50,  # Tamaño por estudiantes
            c=plot_data['ew_potential_score'],
            cmap='RdYlGn',
            alpha=0.7,
            edgecolors='black',
            linewidth=1
        )
        
        # Zona ideal
        ax.axhspan(10, plot_data['grade_std'].max() + 5, xmin=0.2, xmax=0.8, 
                   alpha=0.15, color='green', label='Zona ideal')
        ax.axhline(y=10, color='green', linestyle='--', alpha=0.5)
        ax.axvline(x=0.2, color='green', linestyle='--', alpha=0.5)
        ax.axvline(x=0.8, color='green', linestyle='--', alpha=0.5)
        
        # Etiquetas para top cursos
        top_5 = plot_data.nlargest(5, 'ew_potential_score')
        for _, row in top_5.iterrows():
            ax.annotate(
                row['course_name'][:20],
                (row['pass_rate'], row['grade_std']),
                fontsize=8, alpha=0.8,
                xytext=(5, 5), textcoords='offset points'
            )
        
        ax.set_xlabel('Tasa de Aprobación (Pass Rate)', fontsize=11)
        ax.set_ylabel('Desviación Estándar de Notas', fontsize=11)
        ax.set_title('🎯 Identificación de Cursos con Alto Potencial para Early Warning', 
                     fontsize=13, fontweight='bold')
        
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label('EW Potential Score', fontsize=10)
        
        ax.legend(loc='upper right')
        
        plt.tight_layout()
        plt.savefig(f'{fig_dir}viz_02_course_scatter.png', dpi=150, bbox_inches='tight')
        plt.show()
        print(f"✅ Guardado: {fig_dir}viz_02_course_scatter.png")
else:
    print("⚠️ No hay datos suficientes para scatter plot")

In [ ]:
# 6.3 Top 15 Cursos por EW Potential Score
if not courses_df.empty and 'ew_potential_score' in courses_with_metrics.columns:
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Top 15 cursos
    top_courses = courses_with_metrics.nlargest(15, 'ew_potential_score')
    
    if len(top_courses) > 0:
        # Crear labels más legibles
        labels = [f"{row['course_id']}: {row['course_name'][:35]}" 
                  for _, row in top_courses.iterrows()]
        
        colors = plt.cm.RdYlGn(top_courses['ew_potential_score'] / top_courses['ew_potential_score'].max())
        
        bars = ax.barh(range(len(top_courses)), top_courses['ew_potential_score'], color=colors)
        ax.set_yticks(range(len(top_courses)))
        ax.set_yticklabels(labels, fontsize=9)
        ax.invert_yaxis()
        
        ax.set_xlabel('EW Potential Score', fontsize=11)
        ax.set_title('🏆 Top 15 Cursos por Potencial para Early Warning', 
                     fontsize=13, fontweight='bold')
        
        # Agregar valores
        for bar, score in zip(bars, top_courses['ew_potential_score']):
            ax.text(score + 0.01, bar.get_y() + bar.get_height()/2,
                    f'{score:.3f}', va='center', fontsize=9)
        
        plt.tight_layout()
        plt.savefig(f'{fig_dir}viz_03_top_courses_ranking.png', dpi=150, bbox_inches='tight')
        plt.show()
        print(f"✅ Guardado: {fig_dir}viz_03_top_courses_ranking.png")
else:
    print("⚠️ No hay datos suficientes para ranking")

In [ ]:
# 6.4 Actividad vs Resultados Académicos
if not courses_df.empty and 'avg_page_views' in courses_with_metrics.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    plot_data = courses_with_metrics[
        courses_with_metrics['avg_page_views'].notna() & 
        courses_with_metrics['pass_rate'].notna()
    ].copy()
    
    if len(plot_data) > 0:
        # Plot 1: Page Views vs Pass Rate
        ax1 = axes[0]
        ax1.scatter(
            plot_data['avg_page_views'],
            plot_data['pass_rate'],
            s=100, alpha=0.6, c='steelblue', edgecolors='black'
        )
        ax1.set_xlabel('Promedio Page Views por Estudiante', fontsize=11)
        ax1.set_ylabel('Tasa de Aprobación', fontsize=11)
        ax1.set_title('📈 Actividad vs Aprobación', fontsize=12, fontweight='bold')
        
        # Línea de tendencia
        if len(plot_data) > 2:
            z = np.polyfit(plot_data['avg_page_views'], plot_data['pass_rate'], 1)
            p = np.poly1d(z)
            x_line = np.linspace(plot_data['avg_page_views'].min(), 
                                 plot_data['avg_page_views'].max(), 100)
            ax1.plot(x_line, p(x_line), 'r--', alpha=0.8, label='Tendencia')
            ax1.legend()
        
        # Plot 2: Participations vs Grade Mean
        ax2 = axes[1]
        plot_data2 = courses_with_metrics[
            courses_with_metrics['avg_participations'].notna() & 
            courses_with_metrics['grade_mean'].notna()
        ]
        if len(plot_data2) > 0:
            ax2.scatter(
                plot_data2['avg_participations'],
                plot_data2['grade_mean'],
                s=100, alpha=0.6, c='coral', edgecolors='black'
            )
            ax2.set_xlabel('Promedio Participaciones por Estudiante', fontsize=11)
            ax2.set_ylabel('Nota Promedio del Curso', fontsize=11)
            ax2.set_title('💬 Participación vs Rendimiento', fontsize=12, fontweight='bold')
            
            if len(plot_data2) > 2:
                z = np.polyfit(plot_data2['avg_participations'], plot_data2['grade_mean'], 1)
                p = np.poly1d(z)
                x_line = np.linspace(plot_data2['avg_participations'].min(), 
                                     plot_data2['avg_participations'].max(), 100)
                ax2.plot(x_line, p(x_line), 'r--', alpha=0.8, label='Tendencia')
                ax2.legend()
        
        plt.tight_layout()
        plt.savefig(f'{fig_dir}viz_04_activity_vs_grades.png', dpi=150, bbox_inches='tight')
        plt.show()
        print(f"✅ Guardado: {fig_dir}viz_04_activity_vs_grades.png")
else:
    print("⚠️ No hay datos de actividad para visualizar")

In [ ]:
# 6.5 Heatmap: LMS Design vs Outcomes (Top 10 cursos)
if not courses_df.empty and len(courses_with_metrics) >= 3:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Top 10 cursos por potencial
    top_10 = courses_with_metrics.nlargest(10, 'ew_potential_score')
    
    # Métricas para heatmap
    heatmap_cols = ['n_assignments', 'n_modules', 'n_quizzes', 
                    'avg_page_views', 'avg_participations', 'pass_rate', 'grade_std']
    available_cols = [c for c in heatmap_cols if c in top_10.columns]
    
    if len(available_cols) >= 3:
        heatmap_data = top_10[available_cols].copy()
        
        # Normalizar cada columna 0-1
        for col in available_cols:
            max_val = heatmap_data[col].max()
            if max_val > 0:
                heatmap_data[col] = heatmap_data[col] / max_val
        
        heatmap_data.index = [f"{row['course_id']}: {row['course_name'][:25]}" 
                              for _, row in top_10.iterrows()]
        
        sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='YlOrRd',
                    linewidths=0.5, ax=ax,
                    cbar_kws={'label': 'Valor Normalizado (0-1)'})
        
        ax.set_title('🔥 Perfil de Cursos Top 10 (Normalizado)', 
                     fontsize=13, fontweight='bold')
        ax.set_xlabel('Métricas', fontsize=11)
        ax.set_ylabel('Curso', fontsize=11)
        
        plt.tight_layout()
        plt.savefig(f'{fig_dir}viz_05_course_heatmap.png', dpi=150, bbox_inches='tight')
        plt.show()
        print(f"✅ Guardado: {fig_dir}viz_05_course_heatmap.png")
else:
    print("⚠️ No hay suficientes cursos para heatmap")

## 7. Análisis de Correlaciones entre Métricas

In [ ]:
# 7.1 Matriz de correlación entre métricas de curso
if not courses_df.empty and len(courses_with_metrics) > 5:
    # Seleccionar métricas numéricas relevantes
    corr_cols = ['total_students', 'n_students_with_grades', 'grade_std', 'grade_mean',
                 'pass_rate', 'n_assignments', 'n_modules', 'n_quizzes',
                 'avg_page_views', 'avg_participations', 'avg_on_time_rate', 'avg_missing_rate']
    
    available_corr_cols = [c for c in corr_cols if c in courses_with_metrics.columns]
    
    corr_data = courses_with_metrics[available_corr_cols].dropna(how='all', axis=1)
    
    if len(corr_data.columns) >= 4:
        fig, ax = plt.subplots(figsize=(12, 10))
        
        corr_matrix = corr_data.corr()
        
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        
        sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f',
                    cmap='RdBu_r', center=0, vmin=-1, vmax=1,
                    square=True, linewidths=1, ax=ax,
                    cbar_kws={'label': 'Correlación'})
        
        ax.set_title('🔗 Matriz de Correlación entre Métricas de Curso', 
                     fontsize=13, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(f'{fig_dir}viz_06_correlation_matrix.png', dpi=150, bbox_inches='tight')
        plt.show()
        print(f"✅ Guardado: {fig_dir}viz_06_correlation_matrix.png")
        
        # Mostrar correlaciones más fuertes
        print("\n📊 CORRELACIONES MÁS FUERTES (|r| > 0.5)")
        print("=" * 50)
        corr_pairs = []
        cols = list(corr_matrix.columns)
        for i in range(len(cols)):
            for j in range(i+1, len(cols)):
                r = corr_matrix.iloc[i, j]
                if abs(r) > 0.5:
                    corr_pairs.append({
                        'Variables': f"{cols[i]} ↔ {cols[j]}",
                        'r': r
                    })
        
        if corr_pairs:
            corr_pairs_df = pd.DataFrame(corr_pairs)
            corr_pairs_df = corr_pairs_df.sort_values('r', key=abs, ascending=False)
            display(corr_pairs_df.round(3))
        else:
            print("No se encontraron correlaciones fuertes (|r| > 0.5)")
else:
    print("⚠️ No hay suficientes cursos para análisis de correlación")

In [ ]:
# 7.2 Análisis: ¿Qué predice el éxito estudiantil?
if not courses_df.empty and 'pass_rate' in courses_with_metrics.columns:
    # Correlaciones con pass_rate
    numeric_cols = courses_with_metrics.select_dtypes(include=[np.number]).columns
    
    correlations_with_pass = {}
    for col in numeric_cols:
        if col != 'pass_rate':
            valid_data = courses_with_metrics[[col, 'pass_rate']].dropna()
            if len(valid_data) > 3:
                r = valid_data[col].corr(valid_data['pass_rate'])
                if not np.isnan(r):
                    correlations_with_pass[col] = r
    
    if correlations_with_pass:
        print("📊 CORRELACIONES CON TASA DE APROBACIÓN")
        print("=" * 50)
        print("(Positivo = más de esta métrica → más aprobación)")
        print()
        
        sorted_corr = sorted(correlations_with_pass.items(), key=lambda x: abs(x[1]), reverse=True)
        
        for var, r in sorted_corr[:10]:
            direction = "↑" if r > 0 else "↓"
            strength = "fuerte" if abs(r) > 0.5 else "moderada" if abs(r) > 0.3 else "débil"
            print(f"  {var}: r = {r:+.3f} ({direction} {strength})")
else:
    print("⚠️ No hay datos de pass_rate para análisis")

## 8. Ranking Final de Cursos para Early Warning

In [ ]:
# Crear ranking final
if not courses_df.empty and 'ew_potential_score' in courses_with_metrics.columns:
    # Ordenar por score
    ranked_courses = courses_with_metrics.sort_values('ew_potential_score', ascending=False).copy()
    ranked_courses['rank'] = range(1, len(ranked_courses) + 1)
    
    print("🏆 RANKING FINAL DE CURSOS PARA EARLY WARNING")
    print("=" * 70)
    print("\nCriterios de Ranking:")
    print("  • Grade Variance (30%): Mayor varianza = mejor predicción")
    print("  • Pass Rate Score (20%): Ideal cerca de 50%")
    print("  • LMS Design (20%): Más recursos = más señales")
    print("  • Activity (15%): Engagement estudiantil")
    print("  • Data Quality (15%): Completitud de datos")
    print()
    
    # Tabla de ranking
    ranking_cols = ['rank', 'course_id', 'course_name', 'total_students', 
                    'grade_std', 'pass_rate', 'n_assignments', 'ew_potential_score', 'recommendation']
    available_cols = [c for c in ranking_cols if c in ranked_courses.columns]
    
    display(ranked_courses[available_cols].head(15).round(3))
    
    # Estadísticas
    print("\n📊 ESTADÍSTICAS DEL RANKING")
    print("=" * 50)
    print(f"Total cursos analizados: {len(ranked_courses)}")
    print(f"Cursos HIGH POTENTIAL: {(ranked_courses['recommendation'] == 'HIGH POTENTIAL').sum()}")
    print(f"Cursos MEDIUM POTENTIAL: {(ranked_courses['recommendation'] == 'MEDIUM POTENTIAL').sum()}")
    print(f"Promedio EW Score: {ranked_courses['ew_potential_score'].mean():.3f}")
else:
    print("⚠️ No hay datos para crear ranking")

In [ ]:
# Visualización del ranking final con breakdown de componentes
if not courses_df.empty and 'ew_potential_score' in courses_with_metrics.columns:
    fig, axes = plt.subplots(1, 2, figsize=(16, 10))
    
    # Top 15 para visualización
    top_15 = ranked_courses.head(15)
    
    if len(top_15) > 0:
        # Plot 1: Ranking por EW Potential Score
        ax1 = axes[0]
        labels = [f"#{row['rank']}: {row['course_name'][:30]}" for _, row in top_15.iterrows()]
        colors = plt.cm.RdYlGn(top_15['ew_potential_score'] / top_15['ew_potential_score'].max())
        
        bars = ax1.barh(range(len(top_15)), top_15['ew_potential_score'], color=colors)
        ax1.set_yticks(range(len(top_15)))
        ax1.set_yticklabels(labels, fontsize=9)
        ax1.invert_yaxis()
        ax1.set_xlabel('EW Potential Score', fontsize=11)
        ax1.set_title('🏆 Top 15 Cursos - Ranking Final', fontsize=12, fontweight='bold')
        
        # Agregar valores
        for bar, score in zip(bars, top_15['ew_potential_score']):
            ax1.text(score + 0.005, bar.get_y() + bar.get_height()/2,
                     f'{score:.3f}', va='center', fontsize=8)
        
        # Plot 2: Breakdown de componentes (stacked bar)
        ax2 = axes[1]
        
        components = ['grade_variance_score', 'pass_rate_score', 'lms_design_score', 
                      'activity_score', 'data_quality_score']
        component_labels = ['Grade Variance', 'Pass Rate', 'LMS Design', 'Activity', 'Data Quality']
        component_colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']
        
        available_components = [c for c in components if c in top_15.columns]
        
        if available_components:
            bottom = np.zeros(len(top_15))
            for comp, label, color in zip(components, component_labels, component_colors):
                if comp in top_15.columns:
                    values = top_15[comp].fillna(0).values
                    ax2.barh(range(len(top_15)), values, left=bottom, 
                             label=label, color=color, alpha=0.8)
                    bottom += values
            
            ax2.set_yticks(range(len(top_15)))
            ax2.set_yticklabels([f"#{r}" for r in top_15['rank']], fontsize=9)
            ax2.invert_yaxis()
            ax2.set_xlabel('Score Components', fontsize=11)
            ax2.set_title('📊 Desglose por Componente', fontsize=12, fontweight='bold')
            ax2.legend(loc='lower right', fontsize=8)
        
        plt.tight_layout()
        plt.savefig(f'{fig_dir}viz_07_final_ranking.png', dpi=150, bbox_inches='tight')
        plt.show()
        print(f"✅ Guardado: {fig_dir}viz_07_final_ranking.png")
else:
    print("⚠️ No hay datos para visualizar ranking")

## 9. Insights y Recomendaciones por Curso

In [ ]:
def generate_course_insights(df):
    """
    Genera insights automáticos basados en los cursos analizados.
    """
    insights = []
    
    if df.empty:
        return ["⚠️ No hay datos para generar insights"]
    
    # Top performer
    if 'ew_potential_score' in df.columns:
        top = df.nlargest(1, 'ew_potential_score').iloc[0]
        insights.append(f"🥇 **Mejor curso para EW:** {top['course_name'][:40]} (score: {top['ew_potential_score']:.3f})")
    
    # Curso con mayor varianza de notas
    if 'grade_std' in df.columns:
        high_var = df.nlargest(1, 'grade_std').iloc[0]
        if pd.notna(high_var['grade_std']):
            insights.append(f"📊 **Mayor varianza:** {high_var['course_name'][:40]} (σ={high_var['grade_std']:.1f})")
    
    # Curso con más actividad
    if 'avg_page_views' in df.columns:
        high_activity = df.nlargest(1, 'avg_page_views').iloc[0]
        insights.append(f"🚀 **Más activo:** {high_activity['course_name'][:40]} ({high_activity['avg_page_views']:.0f} views/estudiante)")
    
    # Curso con mejor pass rate en zona ideal
    if 'pass_rate' in df.columns:
        ideal_zone = df[(df['pass_rate'] >= 0.3) & (df['pass_rate'] <= 0.7)]
        if len(ideal_zone) > 0:
            best_ideal = ideal_zone.nlargest(1, 'ew_potential_score').iloc[0]
            insights.append(f"🎯 **Mejor en zona ideal:** {best_ideal['course_name'][:40]} (pass rate: {best_ideal['pass_rate']:.0%})")
    
    # Estadísticas generales
    if 'has_grades' in df.columns:
        pct_with_grades = df['has_grades'].mean() * 100
        insights.append(f"📈 **Cursos con notas:** {pct_with_grades:.0f}% ({df['has_grades'].sum()}/{len(df)})")
    
    # Promedio de recursos
    if 'n_assignments' in df.columns:
        avg_assignments = df['n_assignments'].mean()
        insights.append(f"📝 **Promedio assignments:** {avg_assignments:.1f} por curso")
    
    return insights


if not courses_df.empty:
    print("💡 INSIGHTS PRINCIPALES")
    print("=" * 60)
    
    insights = generate_course_insights(courses_with_metrics)
    for insight in insights:
        print(f"\n{insight}")
else:
    print("⚠️ No hay datos para generar insights")

In [ ]:
# Recomendaciones específicas por curso (Top 5)
def generate_course_recommendations(row):
    """
    Genera recomendaciones específicas para cada curso.
    """
    recs = []
    
    # Baja varianza de notas
    if pd.notna(row.get('grade_std')) and row['grade_std'] < 10:
        recs.append("⚠️ Baja varianza de notas - difícil para predicción")
    
    # Pass rate extremo
    if pd.notna(row.get('pass_rate')):
        if row['pass_rate'] < 0.2:
            recs.append("❌ Pass rate muy bajo (<20%) - revisar dificultad")
        elif row['pass_rate'] > 0.9:
            recs.append("⚠️ Pass rate muy alto (>90%) - poca señal de riesgo")
    
    # Pocos assignments
    if row.get('n_assignments', 0) < 5:
        recs.append("📝 Pocos assignments - agregar más evaluaciones")
    
    # Baja actividad
    if pd.notna(row.get('avg_page_views')) and row['avg_page_views'] < 50:
        recs.append("📉 Baja actividad estudiantil - revisar engagement")
    
    # Alta tasa de missing
    if pd.notna(row.get('avg_missing_rate')) and row['avg_missing_rate'] > 0.3:
        recs.append("⏰ Alta tasa de entregas faltantes ({:.0%})".format(row['avg_missing_rate']))
    
    if not recs:
        recs.append("✅ Buen candidato para Early Warning")
    
    return recs


if not courses_df.empty and 'ew_potential_score' in courses_with_metrics.columns:
    print("📋 RECOMENDACIONES POR CURSO (Top 10)")
    print("=" * 70)
    
    top_10 = courses_with_metrics.nlargest(10, 'ew_potential_score')
    
    for idx, (_, row) in enumerate(top_10.iterrows(), 1):
        recs = generate_course_recommendations(row)
        print(f"\n#{idx} {row['course_name'][:50]}")
        print(f"    Course ID: {row['course_id']} | Score: {row['ew_potential_score']:.3f}")
        for rec in recs:
            print(f"    → {rec}")
else:
    print("⚠️ No hay datos para recomendaciones")

## 10. Exportar Resultados

In [ ]:
# Guardar resultados
output_dir = '../data/analytics/'
os.makedirs(output_dir, exist_ok=True)

if not courses_df.empty and 'ew_potential_score' in courses_with_metrics.columns:
    # CSV con ranking completo
    export_cols = ['rank', 'course_id', 'course_name', 'account_name', 'total_students',
                   'n_students_with_grades', 'grade_mean', 'grade_std', 'pass_rate',
                   'n_assignments', 'n_modules', 'n_quizzes',
                   'avg_page_views', 'avg_participations',
                   'ew_potential_score', 'recommendation']
    available_export = [c for c in export_cols if c in ranked_courses.columns]
    
    ranked_courses[available_export].to_csv(f'{output_dir}course_ranking_results.csv', index=False)
    print(f"✅ Guardado: {output_dir}course_ranking_results.csv")
    
    # JSON con resumen
    summary = {
        'generated_at': datetime.now().isoformat(),
        'term_id': ACTIVE_TERM_ID,
        'accounts_scanned': list(ACCOUNTS_TO_SCAN.keys()),
        'total_courses_analyzed': len(ranked_courses),
        'courses_with_grades': int(ranked_courses['has_grades'].sum()),
        'high_potential_courses': int((ranked_courses['recommendation'] == 'HIGH POTENTIAL').sum()),
        'medium_potential_courses': int((ranked_courses['recommendation'] == 'MEDIUM POTENTIAL').sum()),
        'top_5_courses': ranked_courses.head(5)[['course_id', 'course_name', 'ew_potential_score']].to_dict('records'),
        'insights': insights if 'insights' in dir() else []
    }
    
    with open(f'{output_dir}course_analytics_summary.json', 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2, ensure_ascii=False, default=str)
    print(f"✅ Guardado: {output_dir}course_analytics_summary.json")
    
    # Lista de cursos candidatos para Stage 2 (Page Views ETL)
    candidates = ranked_courses[ranked_courses['recommendation'].str.contains('HIGH|MEDIUM', na=False)]
    candidates_list = candidates[['course_id', 'course_name', 'total_students', 'ew_potential_score']].to_dict('records')
    
    with open(f'{output_dir}stage2_candidates.json', 'w', encoding='utf-8') as f:
        json.dump(candidates_list, f, indent=2, ensure_ascii=False)
    print(f"✅ Guardado: {output_dir}stage2_candidates.json ({len(candidates_list)} candidatos)")

print("\n" + "=" * 60)
print("✅ ANÁLISIS A NIVEL DE CURSO COMPLETADO")
print("=" * 60)

---

## Resumen de Métricas Utilizadas (Nivel de Curso)

### Métricas de Entrada (APIs de Canvas)

| Métrica | API | Descripción |
|---------|-----|-------------|
| `n_students` | Enrollments | Estudiantes matriculados |
| `final_score` | Enrollments | Nota final del estudiante |
| `page_views` | Student Summaries | Vistas de página por estudiante |
| `participations` | Student Summaries | Participaciones por estudiante |
| `tardiness_breakdown` | Student Summaries | on_time, late, missing |
| `n_assignments` | Assignments | Cantidad de tareas/evaluaciones |
| `n_modules` | Modules | Cantidad de módulos |
| `n_quizzes` | Quizzes | Cantidad de cuestionarios |

### Métricas Calculadas

| Métrica | Descripción | Peso en Ranking |
|---------|-------------|-----------------|
| `grade_std` | Desviación estándar de notas | 30% |
| `pass_rate` | Tasa de aprobación (>=57%) | 20% |
| `lms_design_score` | Score de diseño LMS | 20% |
| `activity_score` | Score de actividad | 15% |
| `data_quality_score` | Completitud de datos | 15% |
| `ew_potential_score` | **Score final para Early Warning** | - |

### Criterios de Recomendación

| Recomendación | Criterios |
|---------------|-----------|
| HIGH POTENTIAL | grade_std > 10, 5+ assignments, pass_rate 20-80% |
| MEDIUM POTENTIAL | grade_std > 10, 3+ assignments |
| LOW | grade_std < 10 o pocos assignments |
| SKIP | Sin datos de notas |

---

*Notebook modificado para análisis a nivel de CURSO*  
*Proyecto Early Warning System - U. Autónoma de Chile*  
*Última actualización: Diciembre 2025*